In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from decisiontree import DecisionTreeModel as DT, accuracy_score 

## Feature -- 2 previous word and 2 after words around the Target
### (Best Feature)

In [ ]:
X_train = pd.read_csv('X_train_range2.csv')
y_train = pd.read_csv('y_train_range2.csv')

In [ ]:
X_dev = pd.read_csv('X_dev_range2.csv')
y_dev = pd.read_csv('y_dev_range2.csv')['Target']

In [ ]:
X_test = pd.read_csv('X_test_range2.csv')
y_test = pd.read_csv('y_test_range2.csv')['Target']

In [ ]:
model1 = DT()

In [ ]:
model1.fit(X_train,y_train)

In [ ]:
y_dev_pred = model1.predict(X_dev)

In [ ]:
acc = accuracy_score(y_dev, y_dev_pred)
acc

In [ ]:
# def accuracy_score(y_true, y_pred):
#     accuracy = np.sum(y_true == y_pred) / len(y_true)
#     return accuracy

In [ ]:
# y_test = y_test['Target']

## Feature -- 1 previous word

In [ ]:
# get target around words list
def _splitFeatures(sent, posi, rang):
    sent = str(sent).split(" ")
    str_len = len(sent)
    list = [sent[int(posi)-i] if int(posi)-i >=0 else "<BEGIN_SENTENCE>" for i in range(rang, 0, -1)] \
            + [sent[int(posi)+1+i] if int(posi)+1+i < str_len else "<END_SENTENCE>" for i in range(rang)]
    return list

In [ ]:
# get one hot encoding dict
def _onehot_map(vocabs):
    df = pd.get_dummies(pd.Series(vocabs))
    df['one_hot'] = df.apply(lambda x: ','.join(x.astype(str)), axis=1)
    r = dict(zip(vocabs, df.one_hot))
    return r

In [ ]:
# clean up raw data
def _check_position(sent, posi):
    sent = str(sent).split(" ")
    index = -1
    if '_______' in sent:
        index = sent.index('_______')
    return posi == index

In [ ]:
# create dataset
def _create_dataset(file, rang, vocabs_oh=None):
    # get file content into DF
    df = pd.DataFrame(open(file, "r").readlines(), columns = ['ORG_DATA'])

    # split into 3 columns
    df[['Target', 'Position', 'Sentence']] = df.ORG_DATA.apply(lambda x: pd.Series(str(x).split(" ", 2)))
    
    # clean up
    df['check'] = df.apply(lambda x : _check_position(x['Sentence'], int(x['Position'])), axis = 1)
    df = df[df['check']]
    
    # get words list around target for each sample
    df['Words_List'] = df.apply(lambda x : _splitFeatures(x['Sentence'], x['Position'], rang), axis = 1)
    
    # create vocabulary using Train dataset. Re-use the vovabulary for Dev and Test dataset
    if vocabs_oh == None:
        # get vocabulary(unique words)
        vocabs = df['Words_List'].explode().unique()
        vocabs = np.append(vocabs, '<UNKNOWN_WORD>')
        # one hot mapping based on vocabs
        vocabs_oh = _onehot_map(vocabs)
    
    # replace each word with one hot mapping
    X = pd.DataFrame(df.Words_List.tolist(), index = df.index)
    X.columns = range(rang*2)  
    X = X.applymap(lambda i: vocabs_oh[i] if i in vocabs_oh else vocabs_oh['<UNKNOWN_WORD>'])
    
    # Generate one hot mapped feature dataframe
    temp = X.apply(lambda x: ','.join(x.astype(str)), axis=1)
    Xs = temp.apply(lambda x: pd.Series(str(x).split(",")))
    
    y = df['Target']
    
    return Xs, y, vocabs_oh

In [ ]:
model2.fit(X_micro,y_micro)

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
y_test = y_test['Target']

In [ ]:
acc = accuracy_score(y_test, y_pred)

In [ ]:
acc